In [1]:
import numpy as np
import pandas as pd
import copy
import keras
from keras.layers import Activation
from keras.layers import Conv2D, BatchNormalization, Dense, Flatten, Reshape
from sklearn.model_selection import train_test_split

# Load the data

In [2]:
def get_data(file): 

    data = pd.read_csv(file)

    feat_raw = data['puzzle']
    label_raw = data['solution']

    feat = []
    label = []

    for i in feat_raw:
    
        x = np.array([int(j) for j in i]).reshape((9,9,1))
        feat.append(x)
    
    feat = np.array(feat)
    feat = feat/9
    feat -= .5    
    
    for i in label_raw:
    
        x = np.array([int(j) for j in i]).reshape((81,1)) - 1
        label.append(x)   
    
    label = np.array(label)
    
    del(feat_raw)
    del(label_raw)    

    x_train, x_test, y_train, y_test = train_test_split(feat, label, test_size=0.2, random_state=42)
    
    return x_train, x_test, y_train, y_test

In [3]:
x_train, x_test, y_train, y_test = get_data('sudoku.csv')


# Train model

In [4]:
def define_model():

    model = keras.models.Sequential()

    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

    model.add(Flatten())
    model.add(Dense(81*9))
    model.add(Reshape((-1, 9)))
    model.add(Activation('softmax'))
    
    return model

In [5]:
model = define_model()
model.compile(loss='sparse_categorical_crossentropy')

model.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
225000/225000 [==============================] - 11318s 50ms/step - loss: 0.3923
Epoch 2/2
225000/225000 [==============================] - 11409s 51ms/step - loss: 0.3823


In [9]:
model.save("trained.h5")
